<a href="https://colab.research.google.com/github/UrukHan/Test/blob/master/test_lab_DS.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Тестовое задание Лаборатории больших данных

Загрузите данные из файла 

In [1]:
from google.colab import files

uploaded = files.upload()

for fn in uploaded.keys():
  print('User uploaded file "{name}" with length {length} bytes'.format(
      name=fn, length=len(uploaded[fn])))

Saving test_set_stockouts.csv to test_set_stockouts.csv
User uploaded file "test_set_stockouts.csv" with length 1266581 bytes


В файле содержится таблица данных. Это временные ряды продаж и остатков нескольких видов продукции.

- ID - идентификатор продукта
- dDate - дата записи
- sales - объем продаж за дату
- rests  - остатки на складе на конец дня

Задание
1. Выявить участки, на которых остатки меньше 100 штук (стокауты)
2. Вывести графики для 5-10 товаров и выделить цветом эти участки
3. * Найти даты поступления новой партии товара и проверить гипотезу о том, что "продажи после поступления компенсируют период стокаута".


In [2]:
import pandas as pd

In [91]:
allData = pd.read_csv('/content/test_set_stockouts.csv')
allData.iloc[:3]

,Unnamed: 0,SAP_Code,dDate,rests,id_
0,277579,ITD000000000164431,2017-12-29,3.0,881
1,277580,ITD000000000164431,2018-01-22,3.0,881
2,277581,ITD000000000164431,2018-11-06,NaN,881


In [157]:
allData['dDate'] = pd.to_datetime(allData['dDate'])
allData['dDate'] = allData['dDate'].dt.strftime('%Y,%m,%d')
allData['rests'] = allData['rests'].fillna(0.0)
#type(allData['dDate'][1])

In [158]:
remains = allData.sort_values(['id_', 'dDate'], ascending = [True, True])
remains.iloc[:5]

,Unnamed: 0,SAP_Code,dDate,rests,id_
0,277579,ITD000000000164431,"2017,12,29",3.0,881
1,277580,ITD000000000164431,"2018,01,22",3.0,881
2,277581,ITD000000000164431,"2018,11,06",0.0,881
3,368893,ITD000000000201818,"2017,09,01",84.0,1391
4,368894,ITD000000000201818,"2017,09,03",84.0,1391


In [159]:
len(remains["id_"].unique())

100

In [160]:
idList = remains.groupby('id_').count().iloc[:,:1].T
idList

id_,881,1391,1394,1427,1781,2211,2496,2826,3146,3678,4373,4543,4696,4741,4804,5126,5238,5506,5716,5864,6118,6159,6520,6555,6865,7534,7966,8093,8293,8570,8707,8891,9730,11330,11410,11582,11785,11837,12131,12213,...,18377,18596,18613,18676,18736,19261,19326,19548,19814,20537,20764,20889,20908,20970,21683,22148,23268,23657,24278,24351,24582,24979,25000,25695,26336,26490,26763,26927,27759,27912,28058,28261,28895,29476,29621,29970,30022,30057,30104,30193
Unnamed: 0,3,590,666,11,85,613,3,814,75,19,53,71,89,58,10,300,208,152,63,173,694,51,287,395,218,115,264,391,794,247,20,121,3,168,47,25,288,92,127,216,...,478,326,142,403,385,356,86,127,605,220,571,428,249,117,647,74,424,372,277,277,304,85,203,254,294,51,130,218,303,363,292,122,203,52,275,216,116,158,181,139


In [161]:
valuesCount = idList.values[0]
valuesCount

array([  3, 590, 666,  11,  85, 613,   3, 814,  75,  19,  53,  71,  89,
        58,  10, 300, 208, 152,  63, 173, 694,  51, 287, 395, 218, 115,
       264, 391, 794, 247,  20, 121,   3, 168,  47,  25, 288,  92, 127,
       216,  48,  11, 115, 826,  79, 394, 337,  51,  91, 346, 192,  75,
       418, 670, 332, 164, 454, 351, 744, 511, 478, 326, 142, 403, 385,
       356,  86, 127, 605, 220, 571, 428, 249, 117, 647,  74, 424, 372,
       277, 277, 304,  85, 203, 254, 294,  51, 130, 218, 303, 363, 292,
       122, 203,  52, 275, 216, 116, 158, 181, 139])

In [168]:
allData.loc[(remains['id_'] ==  1391)].iloc[100:120]

,Unnamed: 0,SAP_Code,dDate,rests,id_
103,368993,ITD000000000201818,"2018,02,25",8.0,1391
104,368994,ITD000000000201818,"2018,02,26",8.0,1391
105,368995,ITD000000000201818,"2018,02,28",8.0,1391
106,368996,ITD000000000201818,"2018,03,01",8.0,1391
107,368997,ITD000000000201818,"2018,03,02",8.0,1391
108,368998,ITD000000000201818,"2018,03,05",219.0,1391
109,368999,ITD000000000201818,"2018,03,10",0.0,1391
110,369000,ITD000000000201818,"2018,03,12",0.0,1391
111,369001,ITD000000000201818,"2018,03,14",215.0,1391
112,369002,ITD000000000201818,"2018,03,15",214.0,1391


In [169]:
for i in idList:
  print(i, end = ": ")
  data = allData.loc[(remains['id_'] ==  i)]
  ind = False
  for j in range(idList[i][0]):
    if j == idList[i][0]-1 and data['rests'].values[j] < 100 and ind == True:
      print(' -', data['dDate'].values[j],)
    elif j == idList[i][0]-1 and data['rests'].values[j] < 100:
      print(data['dDate'].values[j])
    elif data['rests'].values[j] < 100 and ind == False:
      print(data['dDate'].values[j], end = '')
      ind = True
    elif data['rests'].values[j] < 100:
      pass
    elif ind == True:
      print(' -', data['dDate'].values[j-1], end = '     ')
      ind = False
    else:
      pass
  print()

881: 2017,12,29 - 2018,11,06

1391: 2017,09,01 - 2018,03,02     2018,03,10 - 2018,03,12     2018,09,13 - 2018,09,13     2018,11,09 - 2019,09,04     
1394: 2017,09,21 - 2017,09,21     2017,09,24 - 2017,09,24     2018,03,06 - 2018,03,13     2018,04,25 - 2018,08,07     2018,09,13 - 2018,09,13     2019,02,06 - 2019,11,12     
1427: 2017,12,14 - 2020,07,30

1781: 2017,09,20 - 2019,07,09

2211: 2017,09,01 - 2018,04,19     2018,08,10 - 2019,04,02     2019,05,21 - 2019,05,21     2019,08,09 - 2020,04,06     
2496: 2017,11,24 - 2020,07,30

2826: 2017,09,01 - 2017,10,16     2018,03,06 - 2018,03,06     2018,03,12 - 2018,03,13     2018,07,22 - 2018,07,22     2018,09,13 - 2018,09,13     2019,03,28 - 2019,03,28     2019,04,05 - 2019,04,12     2019,05,21 - 2019,05,21     2019,10,10 - 2019,10,10     2019,12,11 - 2020,09,03

3146: 2017,09,03 - 2020,07,20

3678: 2017,09,11 - 2019,05,29

4373: 2017,09,06 - 2020,07,29

4543: 2017,09,04 - 2020,07,30

4696: 2018,03,07 - 2019,02,28

4741: 2017,09,06 - 2020,08